In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409
5,1.0,1,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,7.143747
6,0.0,1,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,4.807568
7,1.0,1,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,6.593684
8,1.0,1,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,7.611074
9,0.0,1,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64619
Data columns (total 4 columns):
label                64284 non-null float64
has_date             64284 non-null int64
jieba_cut_content    64284 non-null object
content_sema         64284 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ MB


In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_date,jieba_cut_content,content_sema
18810,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61632,1,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,1,http : / / tengyingge . blogchina . co...,8.289843
5881,1,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26338,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
vectorizer = CountVectorizer()
transfromer = TfidfTransformer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
vectorizer_model = vectorizer.fit(jieba_cut_content)
df0 = vectorizer_model.transform(jieba_cut_content)
transfromer_model = transfromer.fit(df0)
df1 = transfromer_model.transform(df0)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)
data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220013 -0.237482 -0.039927 -0.070134  0.002609 -0.160868   
1  0.014968  0.064740  0.027006 -0.004684  0.008160  0.007396  0.016444   
2  0.011862  0.072033  0.047006  0.001411 -0.002090  0.035907  0.009163   
3  0.019714  0.117438  0.132446  0.060422 -0.103506 -0.074856 -0.012147   
4  0.346257  0.215337 -0.234035 -0.043102 -0.071042  0.000053 -0.162653   

         7         8         9         10        11        12        13  \
0 -0.009831  0.202191 -0.010803 -0.199544  0.039997 -0.063806 -0.428085   
1  0.046042  0.029226 -0.000069  0.007709  0.022971  0.041861  0.011536   
2 -0.012194 -0.016769  0.000228  0.001231 -0.003884 -0.002035  0.003639   
3 -0.005704 -0.027417  0.000429 -0.016901 -0.013466 -0.003897 -0.018067   
4 -0.009149  0.199463 -0.010522 -0.195223  0.038151 -0.060474 -0.416346   

         14        15        16        17        18        19  
0 -0.480897  0.200801  0.094365 -0

In [9]:
# 3.2 数据合并
data['has_date'] = list(x_train['has_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220013 -0.237482 -0.039927 -0.070134  0.002609 -0.160868   
1  0.014968  0.064740  0.027006 -0.004684  0.008160  0.007396  0.016444   
2  0.011862  0.072033  0.047006  0.001411 -0.002090  0.035907  0.009163   
3  0.019714  0.117438  0.132446  0.060422 -0.103506 -0.074856 -0.012147   
4  0.346257  0.215337 -0.234035 -0.043102 -0.071042  0.000053 -0.162653   

          7         8         9      ...             12        13        14  \
0 -0.009831  0.202191 -0.010803      ...      -0.063806 -0.428085 -0.480897   
1  0.046042  0.029226 -0.000069      ...       0.041861  0.011536  0.012740   
2 -0.012194 -0.016769  0.000228      ...      -0.002035  0.003639 -0.002598   
3 -0.005704 -0.027417  0.000429      ...      -0.003897 -0.018067  0.001131   
4 -0.009149  0.199463 -0.010522      ...      -0.060474 -0.416346 -0.471338   

         15        16        17        18        19  has_date  content_sem

In [10]:
%%time
svc = SVC(C = 1, kernel='rbf', degree=3, gamma=0.001)
model = svc.fit(data, y_train)

Wall time: 3min 21s


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(vectorizer_model.transform(jieba_cut_content_test))))
data_test['has_date'] = list(x_test['has_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009051 -0.087589  0.028983  0.000924 -0.032749 -0.157265   
1  0.210902  0.131678 -0.090076 -0.086933 -0.033321  0.039718  0.006933   
2  0.045151  0.277282  0.233913  0.050985 -0.074037 -0.016529  0.011089   
3  0.032907  0.163067  0.113495  0.002429  0.053722  0.008154  0.004540   
4  0.035278  0.065206  0.026091 -0.023774  0.072912 -0.005843 -0.003198   

          7         8         9      ...             12        13        14  \
0  0.008856 -0.134737  0.009726      ...       0.079213 -0.007351 -0.015690   
1 -0.008145  0.011791 -0.000373      ...      -0.025508 -0.081616 -0.056985   
2 -0.015092  0.099227 -0.002327      ...      -0.068826  0.035179 -0.033299   
3 -0.020327  0.117375 -0.002317      ...      -0.062215  0.057261 -0.014190   
4 -0.008197  0.007973  0.000097      ...      -0.001332  0.005372 -0.002638   

         15        16        17        18        19  has_date  content_sem

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("准确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.87646
召回率为:0.98678
F1值为:0.92835
